#Konfiguracja środowiska dla pascal-multi

## Instalacja PyTorch

In [0]:
!pip install -q torchtext==0.2.3

In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

# !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.1-{platform}-linux_x86_64.whl torchvision
# !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
  
import torch

## Instalacja Fast.AI 0.7.0

In [0]:
!git clone https://github.com/fastai/fastai.git
import os
os.chdir("fastai")
!git pull
!git checkout tags/0.7.2 -b v0.7.2

In [0]:
# !pip install matplotlib==2.2.2

In [0]:
os.chdir("..")
!pip install -q https://github.com/fastai/fastai/archive/0.7.0.zip

## ipywidgets fix

In [0]:
!pip install -q ipywidgets==6.0.0

## Instalacja biblioteki PyDrive wrapper & import bibliotek

In [0]:
# This only needs to be done once per notebook.
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

from pathlib import Path
import os

def download_and_save(file_name, file_id):
  downloaded = drive.CreateFile({'id': file_id})
  # fetch file
  downloaded.FetchContent()
  # write file
  with open(file_name,'wb') as f:
       f.write(downloaded.content.read())
      
  print(f'Saved {file_name}')
  
FILE_NAME, FILE_ID = ['file_name', 'file_id']

# Pobranie danych

In [0]:
from pathlib import Path

dataset_path = Path('/content/data/pascal/'); dataset_path
!mkdir -p $dataset_path
!ls -la $dataset_path/

## Pobranie datasetu PASCAL_VOC

In [0]:
# PASCAL_VOC.zip, https://drive.google.com/open?id=1-ZLTyQuZ5biG1uMQgOTZajpqZ7iXuEZD
PASCAL_VOC = {FILE_NAME: 'PASCAL_VOC.zip', FILE_ID: '1-ZLTyQuZ5biG1uMQgOTZajpqZ7iXuEZD'}
PASCAL_VOC_path = dataset_path / PASCAL_VOC[FILE_NAME]

download_and_save(PASCAL_VOC_path, PASCAL_VOC[FILE_ID])

In [0]:
!unzip $PASCAL_VOC_path -d $dataset_path

In [0]:
mv $dataset_path/PASCAL_VOC/*.json $dataset_path

In [0]:
rmdir $dataset_path/PASCAL_VOC

In [0]:
# VOCdevkit_08-Jun-2007.tar, https://drive.google.com/open?id=1wQeb1zYEcXN8PKXQHOCxyvktYeb55IFL
VOCdevkit = {FILE_NAME: 'VOCdevkit_08-Jun-2007.tar', FILE_ID: '1wQeb1zYEcXN8PKXQHOCxyvktYeb55IFL'}
VOCdevkit_path = dataset_path / VOCdevkit[FILE_NAME]
download_and_save(VOCdevkit_path, VOCdevkit[FILE_ID])

In [0]:
!tar -xf $VOCdevkit_path -C $dataset_path

In [0]:
# VOCtrainval_06-Nov-2007.tar, https://drive.google.com/open?id=1yx1w3ly9rxY_EFzfESpGKEVSfP_TlKvF
VOCtrainval = {FILE_NAME: 'VOCtrainval_06-Nov-2007.tar', FILE_ID: '1yx1w3ly9rxY_EFzfESpGKEVSfP_TlKvF'}
VOCtrainval_path = dataset_path / VOCtrainval[FILE_NAME]
download_and_save(VOCtrainval_path, VOCtrainval[FILE_ID])

In [0]:
!tar -xf $VOCtrainval_path -C $dataset_path

In [0]:
ls -lah $dataset_path

## Pobranie przykładowych danych dla prognozowania

In [0]:
# FIXME: przygotować dane z samochodami, te są do innego projektu
# custom_samples.tgz, https://drive.google.com/open?id=1qcanWbaLi2pHA7dJzobwlyVGCRGtzrSt
# custom_samples = {FILE_NAME: 'custom_samples.tgz', FILE_ID: '1qcanWbaLi2pHA7dJzobwlyVGCRGtzrSt'}
# custom_samples_path = dataset_path / custom_samples[FILE_NAME]
# download_and_save(custom_samples_path, custom_samples[FILE_ID])

In [0]:
# !tar -xf $custom_samples_path -C $dataset_path

In [0]:
# ls -lah $dataset_path

## Konfiguracja integracji z Dyskiem Google

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive -o nonempty

# Monitorowanie maszyny wirtualnej

In [0]:
!uptime

In [0]:
!/opt/bin/nvidia-smi

In [0]:
!top -n 1

In [0]:
import os
import psutil

def print_memsize():
  process = psutil.Process(os.getpid())
  print(f'{process.memory_info().rss / 1024**3:.5} GB')

In [0]:
print_memsize()

In [0]:
!ps ax|grep python